In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import copy
from re import sub
from decimal import Decimal
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import geopandas
pd.set_option('display.max_columns', None)


ImportError: cannot import name '_NDFrameIndexer' from 'pandas.core.indexing' (/home/duy/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py)

In [ ]:
def curr_to_dec(money):
    return float(sub(r'[^\d.]', '', money))

In [ ]:
def gdp_preproc(gdp_df):
    gdp_df = gdp_df[['GDP (nominal, 2017)','GDP per capita','Country']]
    gdp_df['GDP (nominal, 2017)'] = gdp_df['GDP (nominal, 2017)'].apply(curr_to_dec)
    gdp_df['GDP per capita'] = gdp_df['GDP per capita'].apply(curr_to_dec)
    return gdp_df
def preproc_col(df):
    return df.rename(columns={"Country,Other":"Country"}).drop(columns=['Unnamed: 0'])
def merge_gdp(df,gdp_df=pd.read_csv("./data/gdp.csv")): 
    return gdp_preproc(gdp_df).merge(df,on='Country')
def add_death_per_cases(df):
    df['Deaths/Cases'] = df['TotalDeaths']/df['TotalCases']
    return df
def duy_preproc(df):
    result = copy.deepcopy(df)
    result = result.drop(df.tail(8).index)    
    result = preproc_col(result)
    result = add_death_per_cases(result)
    result = merge_gdp(result)
    return result

In [ ]:
df = pd.read_csv('./data/2022-04-27.csv')
df = duy_preproc(df)

/tmp/ipykernel_59312/1617424969.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['GDP (nominal, 2017)'] = gdp_df['GDP (nominal, 2017)'].apply(curr_to_dec)
/tmp/ipykernel_59312/1617424969.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['GDP per capita'] = gdp_df['GDP per capita'].apply(curr_to_dec)


In [ ]:
df

,"GDP (nominal, 2017)",GDP per capita,Country,#,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop,Deaths/Cases
0,1.223770e+13,8612.0,China,1.0,207081,1824.0,4876.0,48.0,175431.0,3728.0,26774.0,318.0,144.0,3.0,160000000.0,111163.0,1.439324e+09,Asia,6951.0,295185.0,9.0,1.0,0.03,19.0,0.023546
1,4.872415e+12,38214.0,Japan,16.0,7725765,37258.0,29402.0,58.0,7263359.0,40195.0,433004.0,183.0,61426.0,234.0,46888808.0,372804.0,1.257734e+08,Asia,16.0,4278.0,3.0,296.0,0.50,3443.0,0.003806
2,3.693204e+12,44680.0,Germany,6.0,24488242,124863.0,135369.0,245.0,21885900.0,220700.0,2466973.0,1980.0,290591.0,1606.0,122332384.0,1451664.0,8.427047e+07,Europe,3.0,623.0,1.0,1482.0,3.00,29274.0,0.005528
3,2.650725e+12,1980.0,India,3.0,43068799,3133.0,523693.0,39.0,42528126.0,2563.0,16980.0,698.0,30662.0,373.0,835974079.0,595150.0,1.404643e+09,Asia,33.0,2682.0,2.0,2.0,0.03,12.0,0.012159
4,2.582501e+12,39827.0,France,5.0,28483124,67711.0,145579.0,152.0,26403836.0,158622.0,1933709.0,1677.0,434619.0,2221.0,266484045.0,4066235.0,6.553581e+07,Europe,2.0,450.0,0.0,1033.0,2.00,29506.0,0.005111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,4.276598e+08,4193.0,Tonga,193.0,9838,NaN,11.0,NaN,8721.0,NaN,1106.0,NaN,91202.0,102.0,423475.0,3925790.0,1.078700e+05,Australia/Oceania,11.0,9806.0,0.0,NaN,NaN,10253.0,0.001118
169,3.364275e+08,631.0,Micronesia,227.0,3,NaN,NaN,NaN,1.0,NaN,2.0,NaN,26.0,NaN,NaN,NaN,1.172130e+05,Australia/Oceania,39071.0,NaN,NaN,NaN,NaN,17.0,NaN
170,2.898235e+08,16275.0,Palau,211.0,4486,NaN,6.0,NaN,3913.0,NaN,567.0,1.0,245795.0,329.0,45500.0,2493014.0,1.825100e+04,Australia/Oceania,4.0,3042.0,0.0,NaN,NaN,31067.0,0.001337
171,2.041734e+08,3517.0,Marshall Islands,222.0,17,NaN,NaN,NaN,7.0,NaN,10.0,NaN,284.0,NaN,NaN,NaN,5.991500e+04,Australia/Oceania,3524.0,NaN,NaN,NaN,NaN,167.0,NaN


In [ ]:
df = df.dropna(axis=0,subset=['Deaths/1M pop','Tot\xa0Cases/1M pop'])

In [ ]:
df_1 = df[["GDP per capita","Deaths/Cases"]]

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
ax.set(xscale="log", yscale="log")
sns.scatterplot(x='GDP (nominal, 2017)', y='Deaths/Cases', data=df,ax = ax)

In [ ]:
import pycountry 


In [ ]:
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
df['CODE'] = alpha3code(df['Country'])

In [ ]:
# first let us merge geopandas data with our data
# 'naturalearth_lowres' is geopandas datasets so we can use it directly
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# rename the columns so that we can merge with our data
world.columns=['pop_est', 'continent', 'name', 'CODE', 'gdp_md_est', 'geometry']
# then merge with our data 
merge=pd.merge(world,df,on='CODE')
# last thing we need to do is - merge again with our location data which contains each country’s latitude and longitude
location=pd.read_csv('https://raw.githubusercontent.com/melanieshi0120/COVID-19_global_time_series_panel_data/master/data/countries_latitude_longitude.csv')
merge=merge.merge(location,on='name')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also

    print(world['name'])

In [ ]:
# plot confirmed cases world map 
merge.plot(column='Deaths/Cases', scheme="quantiles",
           figsize=(25, 20),
           legend=True,cmap='Reds')
# add countries names and numbers 

plt.show()

In [ ]:
# plot confirmed cases world map 
merge.plot(column='Tot\xa0Cases/1M pop', scheme="quantiles",
           figsize=(25, 20),
           legend=True,cmap='Reds')
# add countries names and numbers 
for i in range(0,10):
    plt.text(float(merge.longitude[i]),float(merge.latitude[i]),"{}\n{}".format(merge.name[i],merge['Deaths/Cases'][i]),size=10)
plt.show()

In [ ]:

f, ax = plt.subplots(figsize=(7, 7))

sns.regplot(x='Tot\xa0Cases/1M pop', y='Deaths/Cases', data=df,ax = ax)

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
ax.set(xscale="log", yscale="log")
sns.scatterplot(x='GDP (nominal, 2017)', y='Deaths/Cases', data=df,ax = ax)

In [ ]:
w = np.ones(len(df))
w[len(df) * 6 // 10 :] = 2
w

In [ ]:
Y = stats.zscore(df['Deaths/Cases'])
X = stats.zscore(df['Tot\xa0Cases/1M pop'])
model_OLS = sm.OLS(Y,X)
result_OLS = model_OLS.fit()
result_OLS.summary()

In [ ]:
Y_pred = result_OLS.predict(X)

In [ ]:
_std = np.sqrt((Y-Y_pred)**2)
_std

In [ ]:
sns.lineplot(x = X,y = _std)

In [ ]:
from sklearn.cluster import KMeans
X = merge[["Deaths/Cases","Tot\xa0Cases/1M pop"]]
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
merge['label'] = kmeans.labels_

In [ ]:
# plot confirmed cases world map 
merge.plot(column='label', scheme="quantiles",
           figsize=(25, 20),
           legend=True,cmap='seismic')
# add countries names and numbers 
for i in range(0,10):
    plt.text(float(merge.longitude[i]),float(merge.latitude[i]),"{}\n{}".format(merge.name[i],merge['Deaths/Cases'][i]),size=10)
plt.show()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also

    print(merge['Country'])

In [ ]:
df_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

print(f"{type(df_world)}, {df_world.geometry.name}")

print(df_world.head())

print(df_world.geometry.geom_type.value_counts())

In [ ]:
df_world.plot(figsize=(10,6))


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also

    print(df_world)